# Wannier workflow


In [4]:
from aiida import orm
from aiida_workgraph import WorkGraph
from aiida_workgraph.decorator import node
from aiida_quantumespresso.workflows.pw.base import PwBaseWorkChain
from aiida_quantumespresso.calculations.dos import DosCalculation
from aiida_quantumespresso.calculations.projwfc import ProjwfcCalculation
from aiida_wannier90_workflows.workflows import OpenGridBaseWorkChain


wg = WorkGraph()
wg.ctx = {'current_structure': None, 'current_number_of_bands': None, 'parent_folder': None}
# -------- scf -----------
scf_node = wg.nodes.new(PwBaseWorkChain, name='scf')
scf_node.to_ctx = [['remote_folder', 'scf_parent_folder']]
# -------- nscf -----------
nscf_node = wg.nodes.new(PwBaseWorkChain, name='nscf')
nscf_node.set({"pw.parent_folder": scf_node.outputs['remote_folder']})
# -------- opengrid -----------
projwfc1 = wg.nodes.new(OpenGridBaseWorkChain, name='opengrid', 
                          parent_folder=nscf_node.outputs['remote_folder'])
# -------- projwfc -----------
projwfc1 = wg.nodes.new(ProjwfcCalculation, name='projwfc', 
                          parent_folder=nscf_node.outputs['remote_folder'])
wg

NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'WorkGra…